## Import All Required Libraries

In [1]:
# Import all connection libraries

from spotipy import SpotifyClientCredentials
import spotipy
import sqlalchemy as sql
import psycopg2

# Import all data manipulation libraries

import pandas as pd
from pandarallel import pandarallel
import numpy as np
from tqdm import tqdm
pandarallel.initialize(verbose=0, nb_workers=8)

In [2]:
from functools import lru_cache

# Use the lru_cache decorator to cache the result of the function
# maxsize=1000 means the cache will store the result of up to 1000 items
@lru_cache(maxsize=1000)
def get_playlist_tracks(playlist_uri):
    """
    This function uses the Spotify API to get the tracks of a given playlist.
    The function is decorated with the lru_cache decorator to cache the result for each unique playlist_uri.
    So, if the same playlist_uri is passed to the function again, the cached result will be returned
    instead of making a new API call.
    :param playlist_uri: The Spotify URI of the playlist
    :return: A DataFrame containing the tracks of the playlist
    """
    # Use the Spotify API to get the tracks of the playlist
    tracks = pd.json_normalize(sp.playlist_tracks(playlist_uri), record_path=['items'])
    return tracks

## Load Initial Training Data

In [2]:
# establish connection to Spotify API

cid = '3fda75b7146a4769b207ee44017b3abe'
secret = 'aaaa11ed71604ff19d592070be0f6bbd'
client_credentials = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials, requests_timeout=15, retries=10)

# establish connection to Postgres

host_name = 'localhost'
database_name = 'Spotify'
engine = sql.create_engine('postgresql+psycopg2://postgres:DataNerd2023!!\
@localhost/Spotify')

# load training data
new_batch = pd.read_csv('distinct_playlists.csv')[['playlist_uri', 'playlist_name']]
new_batch['playlist_uri'] = new_batch['playlist_uri'].str.strip()

db_query = pd.read_sql('SELECT DISTINCT playlist_uri FROM playlist_tracks', engine)

outer = new_batch.merge(db_query, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

new_batch = pd.DataFrame(anti_join)
new_batch = new_batch[5:10]
new_batch.head()

,playlist_uri,playlist_name
687,spotify:playlist:0Pq0GkVjNxLtZHskNbi1mu,soft pop/sleep
759,spotify:playlist:0rKZdoM1157qlnakn4mXzj,"Work R&B, Pop, & More /Clean"
760,spotify:playlist:0rLWNAXWIyS3NmMcrxVIOD,Morning Workout Vibezz ????
761,spotify:playlist:0RMbt8YFa5ryki24PXKvTl,Blvck; Hip Hop
762,spotify:playlist:0rnBRBTSCQMOxj6h0Y6PS1,Lofi Hip Hop Playlist 2022


In [ ]:
sp.track('spotify:track:4ndLNUrzmNEQJCS2T6kH05')

## Iterate Through Training Data to Change Grain of Data

In [4]:
load_batch = []
series = new_batch['playlist_uri'].to_dict()
for playlist in tqdm(series.values()):
    try:
        tracks = get_playlist_tracks(playlist)
        if len(tracks) >= 100:
                tracks2 = pd.json_normalize(sp.playlist_tracks(playlist, offset=100), record_path=['items'])
                tracks3 = pd.json_normalize(sp.playlist_tracks(playlist, offset=200), record_path=['items'])
                tracks = pd.concat([tracks, tracks2, tracks3])
                secondary_data = pd.json_normalize(sp.playlist(playlist))
                tracks['playlist_uri'] = playlist
                tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
                tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri", "added_by.external_urls.spotify": "added_by_external_urls_spotify", 'added_by.href':"added_by_href", "added_by.id":"added_by_id", "added_by.type":"added_by_type", "added_by.uri":"added_by_uri", "track.album.album_type":"track_album_album_type", "track.album.external_urls.spotify":"track_album_external_urls_spotify", "track.album.href":"track_album_href", "track.album.id":"track_album_id", "track.album.release_date_precision":"track_album_release_date_precision", "track.album.total_tracks":"track_album_total_tracks","track.album.type":"track_album_type", "track.disc_number":"track_disc_number", "track.episode":"track_episode", "track.external_ids.isrc":"track_external_ids_isrc", "track.external_urls.spotify":"track_external_ids_spotify", "track.href":"track_href", "track.id":"track_id", "track.is_local":"track_is_local", "track.popularity":"track_popularity", "track.preview_url":"track_preview_url", "track.track":"track_track", "track.track_number":"track_track_number", "track.type":"track_type", "video_thumbnail.url":"video_thumbnail_url", "external_urls.spotify":"external_urls_spotify", "followers.href":"followers_href", "followers.total":"followers_total", "owner.display_name":"owner_display_name", "owner.external_urls.spotify":"owner_external_urls_spotify", "owner.href":"owner_href", "owner.id":"owner_id", "owner.type":"owner_type", "owner.uri":"owner_uri", "tracks.href":"tracks_href", "tracks.limit":"tracks_limit", "tracks.next":"tracks_next", "tracks.offset":"tracks_offset", "tracks.previous":"tracks_previous", "tracks.total":"tracks_total"})
                tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])

                load_batch.append(tracks)
        else:
                secondary_data = pd.json_normalize(sp.playlist(playlist))
                tracks['playlist_uri'] = playlist
                tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
                tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri", "added_by.external_urls.spotify": "added_by_external_urls_spotify", 'added_by.href':"added_by_href", "added_by.id":"added_by_id", "added_by.type":"added_by_type", "added_by.uri":"added_by_uri", "track.album.album_type":"track_album_album_type", "track.album.external_urls.spotify":"track_album_external_urls_spotify", "track.album.href":"track_album_href", "track.album.id":"track_album_id", "track.album.release_date_precision":"track_album_release_date_precision", "track.album.total_tracks":"track_album_total_tracks","track.album.type":"track_album_type", "track.disc_number":"track_disc_number", "track.episode":"track_episode", "track.external_ids.isrc":"track_external_ids_isrc", "track.external_urls.spotify":"track_external_ids_spotify", "track.href":"track_href", "track.id":"track_id", "track.is_local":"track_is_local", "track.popularity":"track_popularity", "track.preview_url":"track_preview_url", "track.track":"track_track", "track.track_number":"track_track_number", "track.type":"track_type", "video_thumbnail.url":"video_thumbnail_url", "external_urls.spotify":"external_urls_spotify", "followers.href":"followers_href", "followers.total":"followers_total", "owner.display_name":"owner_display_name", "owner.external_urls.spotify":"owner_external_urls_spotify", "owner.href":"owner_href", "owner.id":"owner_id", "owner.type":"owner_type", "owner.uri":"owner_uri", "tracks.href":"tracks_href", "tracks.limit":"tracks_limit", "tracks.next":"tracks_next", "tracks.offset":"tracks_offset", "tracks.previous":"tracks_previous", "tracks.total":"tracks_total"})
                tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])
                load_batch.append(tracks)
    except:
            pass
load_batch = pd.concat(load_batch)
load_batch.head()

100%|██████████| 5/5 [00:03<00:00,  1.27it/s]


,added_at,is_local,primary_color_x,added_by_external_urls_spotify,added_by_href,added_by_id,added_by_type,added_by_uri,track_album_album_type,track_album_external_urls_spotify,...,owner_type,owner_uri,tracks_href,tracks_limit,tracks_next,tracks_offset,tracks_previous,tracks_total,track,video_thumbnail
0,2021-01-19T11:18:23Z,False,None,https://open.spotify.com/user/rogsypimentel,https://api.spotify.com/v1/users/rogsypimentel,rogsypimentel,user,spotify:user:rogsypimentel,album,https://open.spotify.com/album/7e7t0MCrNDcJZsP...,...,user,spotify:user:rogsypimentel,https://api.spotify.com/v1/playlists/0Ri0USu1Y...,100,https://api.spotify.com/v1/playlists/0Ri0USu1Y...,0,None,137,NaN,NaN
1,2021-01-19T11:18:36Z,False,None,https://open.spotify.com/user/rogsypimentel,https://api.spotify.com/v1/users/rogsypimentel,rogsypimentel,user,spotify:user:rogsypimentel,album,https://open.spotify.com/album/41GuZcammIkupMP...,...,user,spotify:user:rogsypimentel,https://api.spotify.com/v1/playlists/0Ri0USu1Y...,100,https://api.spotify.com/v1/playlists/0Ri0USu1Y...,0,None,137,NaN,NaN
2,2021-01-19T11:18:40Z,False,None,https://open.spotify.com/user/rogsypimentel,https://api.spotify.com/v1/users/rogsypimentel,rogsypimentel,user,spotify:user:rogsypimentel,album,https://open.spotify.com/album/2nkto6YNI4rUYTL...,...,user,spotify:user:rogsypimentel,https://api.spotify.com/v1/playlists/0Ri0USu1Y...,100,https://api.spotify.com/v1/playlists/0Ri0USu1Y...,0,None,137,NaN,NaN
3,2021-01-19T11:18:44Z,False,None,https://open.spotify.com/user/rogsypimentel,https://api.spotify.com/v1/users/rogsypimentel,rogsypimentel,user,spotify:user:rogsypimentel,album,https://open.spotify.com/album/42WVQWuf1teDysX...,...,user,spotify:user:rogsypimentel,https://api.spotify.com/v1/playlists/0Ri0USu1Y...,100,https://api.spotify.com/v1/playlists/0Ri0USu1Y...,0,None,137,NaN,NaN
4,2021-01-19T11:19:50Z,False,None,https://open.spotify.com/user/rogsypimentel,https://api.spotify.com/v1/users/rogsypimentel,rogsypimentel,user,spotify:user:rogsypimentel,album,https://open.spotify.com/album/6OQ9gBfg5EXeNAE...,...,user,spotify:user:rogsypimentel,https://api.spotify.com/v1/playlists/0Ri0USu1Y...,100,https://api.spotify.com/v1/playlists/0Ri0USu1Y...,0,None,137,NaN,NaN


## Upload Training Data to playlist_tracks Table

In [5]:
load_batch.to_sql('playlist_tracks', engine, if_exists='append')

300

## Identify All Artists in Every Song

In [6]:
df = pd.read_sql('''SELECT DISTINCT track_uri FROM playlist_tracks''', engine)
outer = df.merge(load_batch, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

df = pd.Series(anti_join['track_uri'].head(10000))
base_list = []
for track in tqdm(df):
    try:
        df2 = pd.json_normalize(sp.track(track), record_path=['artists'])
        df2['track_uri'] = track
        df2 = df2[['name', 'track_uri']]
        base_list.append(df2)
    except:
        pass
df2 = pd.concat(base_list)
df2

  0%|          | 0/10000 [00:00<?, ?it/s]

## Get Row Numbers for Pivoting

In [ ]:
df2['RN'] = df2.groupby("track_uri")["name"].rank(method="first", ascending=True)
df2

,name,track_uri,RN
0,Dorrough Music,spotify:track:3lwMGZ9M5Gq4usaZZLV4NK,1.0
0,Arman Cekin,spotify:track:364vyXytDCU4KVgjrDmVQX,1.0
1,Esther Sparkes,spotify:track:364vyXytDCU4KVgjrDmVQX,2.0
0,LoCraft,spotify:track:2ZTvUeTWseCBVhPUfJsgGt,1.0
0,chromonicci,spotify:track:0zARNJ9ss6SpaDUzxB95j0,1.0
...,...,...,...
0,Flavour,spotify:track:7u8pI0Q3NLg2RWvC1lyaGm,1.0
0,$uicideboy$,spotify:track:3cQPyVlnrmmWU5fQJxl0q9,1.0
0,First Aid Kit,spotify:track:3fy2ILYi8upEuracr84cx0,1.0
0,Jason Aldean,spotify:track:5eH1ktZDHgegpkYJOY7tme,1.0


## Pivot DataFrame to Fit all Artists in DataFrame

In [ ]:
df2 = df2.pivot(index='track_uri', columns=['RN'], values='name').reset_index()
df2 = df2.rename(columns={1.0:'artist1', 2.0:'artist2', 3.0:'artist3', 4.0:'artist4', 5.0:'artist5'})
df2 = df2[['track_uri', 'artist1', 'artist2', 'artist3', 'artist4', 'artist5']]
df2

RN,track_uri,artist1,artist2,artist3,artist4,artist5
0,spotify:track:000VUaYhw19g0d6tCGZkJB,6LACK,Dreamville,JID,Lute,NaN
1,spotify:track:005lwxGU1tms6HGELIcUv9,Katy Perry,NaN,NaN,NaN,NaN
2,spotify:track:00AxNl4D4jHL2AEf1W55j5,Dierks Bentley,NaN,NaN,NaN,NaN
3,spotify:track:00EFWaqXnHZ5smJNsHtnGV,HALIENE,Seven Lions,NaN,NaN,NaN
4,spotify:track:00I0pcNkN3IOX3fsYbaB4N,Takeoff,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
9990,spotify:track:7zhyWEAvCMi6W440bAkCTO,Colyn,NaN,NaN,NaN,NaN
9991,spotify:track:7zjzu90Q4mtNNaE9Ol9Zbv,AC/DC,NaN,NaN,NaN,NaN
9992,spotify:track:7zpBRUYHP0RWncVUX9mvJO,Drowning Pool,NaN,NaN,NaN,NaN
9993,spotify:track:7zubSmuSBhEmxJVcOkxvP9,Nonso Amadi,melvitto,NaN,NaN,NaN


## Upload All Artist Data to Database

In [ ]:
df2.to_sql('artists', con=engine, if_exists='append')

995

## Get All Audio Features

In [ ]:
df = pd.read_sql('''SELECT DISTINCT track_uri FROM playlist_tracks''', engine)
outer = df.merge(load_batch, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)
df = pd.Series(anti_join['track_uri'].head(10000))

base_list = []
for track in tqdm(df):
    try:
        df3 = pd.json_normalize(sp.audio_features(track))
        df3['track_uri'] = track
        df3 = df3[['track_uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
        base_list.append(df3)
    except:
        pass
df3 = pd.concat(base_list)
df3

100%|██████████| 10000/10000 [15:55<00:00, 10.47it/s]


,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,spotify:track:3lwMGZ9M5Gq4usaZZLV4NK,0.849,0.5900,0,-8.367,1,0.4280,0.000545,0.000000,0.3430,0.656,180.026,237320,4
0,spotify:track:364vyXytDCU4KVgjrDmVQX,0.562,0.6900,2,-5.998,0,0.0389,0.036800,0.000066,0.1080,0.227,150.056,209608,4
0,spotify:track:2ZTvUeTWseCBVhPUfJsgGt,0.608,0.0696,0,-19.207,1,0.0457,0.812000,0.943000,0.1080,0.088,159.945,183000,4
0,spotify:track:0zARNJ9ss6SpaDUzxB95j0,0.514,0.7670,9,-9.479,1,0.2050,0.375000,0.882000,0.3030,0.718,81.146,205468,4
0,spotify:track:6IanFWu96WUD1pfTirhZBe,0.845,0.8040,10,-5.009,1,0.0609,0.041700,0.859000,0.0749,0.652,125.015,262019,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,spotify:track:7u8pI0Q3NLg2RWvC1lyaGm,0.842,0.9170,11,-3.897,0,0.1980,0.115000,0.000002,0.6590,0.539,102.971,215876,4
0,spotify:track:3cQPyVlnrmmWU5fQJxl0q9,0.678,0.7700,6,-3.206,0,0.1450,0.125000,0.000000,0.1840,0.284,75.009,133903,4
0,spotify:track:3fy2ILYi8upEuracr84cx0,0.876,0.6270,8,-6.367,1,0.0565,0.035500,0.000000,0.0958,0.638,116.075,208893,4
0,spotify:track:5eH1ktZDHgegpkYJOY7tme,0.430,0.8410,6,-2.990,1,0.0370,0.130000,0.000000,0.0517,0.489,180.977,183440,4


## Upload Audio Features to Database

In [ ]:
df3.to_sql('audio_features', con=engine, if_exists='append')

991

In [ ]:
all_data = pd.read_sql('''SELECT DISTINCT playlist_uri, playlist_name, owner_uri, owner_display_name, collaborative, description, followers_total
track_uri, track_name, artist1, artist2, artist3, artist4, artist5, album_uri, album_name, release_date, 
added_by_uri, added_at, track_album_total_tracks, af.duration_ms, "isExplicit", track_popularity, 
track_preview_url, video_thumbnail_url, danceability, energy, loudness, key, mode, acousticness, speechiness, 
instrumentalness, liveness, valence, tempo, time_signature
FROM playlist_tracks pt
JOIN artists a ON pt.track_uri = a.track_uri
JOIN audio_features af ON pt.track_uri = af.track_uri;''', engine)
f = open("s3.csv", "w")
f.truncate()
f.close()

all_data.to_csv('s3.csv', mode="w+")